# Country Dashboard Template

**Standardized structure for all country economic dashboards**

This template provides a consistent framework for analyzing countries across:
- Growth (Activity & Output)
- Prices (Inflation & Wages)
- Slack (Output Gap & Capacity)
- Employment (Labor Market)
- Financial Conditions (Credit & Markets)
- Fiscal Conditions (Budget & Debt)

In [ ]:
import pandas as pd
from IPython.display import Markdown, display

from tulip.core.collection import TulipCollection
from tulip.data.bloomberg import BloombergClient as bb
from tulip.data.haver import HaverClient as hc
from tulip.data.gs import GSClient as gs
from tulip.data.fred import get_series_from_fred
from tulip.plots import plot_line, plot_lines, plot_area, plot_bar
from tulip.genai import iris
from tulip.analysis.country_related.analytics import summarize_gs_eco_fct

from tulip_mania_next.columns_framework import columns

import pycountry
import plotly.offline as pyo
pyo.init_notebook_mode(connected=True)

In [ ]:
# ========================================
# COUNTRY PARAMETERS
# ========================================
# Modify these for each country

# Country identifiers
ctry_iso2 = "US"          # ISO 2-letter code (US, CA, JP, AU, etc.)
ctry_iso3 = "USA"         # ISO 3-letter code
ctry_name = "United States"
ccy = "USD"               # Currency code

# Data source identifiers (vary by provider)
goldman_ctry_id = "US"    # Goldman Sachs geography ID
bloomberg_ctry_prefix = "US"  # Bloomberg country prefix
haver_ctry_suffix = "USECON"  # Haver database suffix

# Get country object for metadata
pyctry = pycountry.countries.get(alpha_2=ctry_iso2)

## 1. Growth: Activity & Output

**Key Questions:**
- How fast is the economy growing?
- What sectors are driving growth?
- What do nowcasts and leading indicators suggest?

### 1.1 Economic Forecasts (Consensus)

In [ ]:
# Goldman Sachs Economic Forecasts
try:
    gs_eco_fct = gs.get_eco_forecast(geographyId=goldman_ctry_id)
    gs_summary = summarize_gs_eco_fct(gs_eco_fct)
    gs_summary = gs_summary[~gs_summary.index.str.contains("ngdp")].to_frame().T
    gs_summary.style.set_caption(f"Goldman Sachs {ctry_name} Economic Forecasts").format(precision=2)
except Exception as e:
    display(Markdown(f"*Goldman Sachs forecasts not available for {ctry_name}: {e}*"))

### 1.2 Growth Nowcasts & Real-Time Indicators

In [ ]:
# Goldman Sachs Current Activity Indicator (CAI)
try:
    cai_series = gs.get_CAI_series(
        geographyId=goldman_ctry_id,
        metricName=["CAI_HEADLINE", "CAI_CONTRIBUTION_TYPE_HARD", "CAI_CONTRIBUTION_TYPE_SOFT"],
        startDate="1980-01-01",
    )
    cai_series = cai_series.set_index("metricName", append=True)["metricValue"].unstack("metricName")
    cai_series.columns = ["Hard", "Soft", "Headline"]
    
    cai_plot = plot_lines(
        cai_series,
        show_0=True,
        title=f"<b>{ctry_name} Current Activity Indicator</b> Updated: {pd.Timestamp.today().strftime('%Y-%m-%d')}",
        years_limit=4,
        tick_suffix="%",
        source="Goldman Sachs",
    )
    cai_plot.show()
except Exception as e:
    display(Markdown(f"*CAI not available for {ctry_name}: {e}*"))

In [ ]:
# Bloomberg Nowcast (if available)
# Common patterns: BENW{ISO2}GC Index or BENW{ISO2}NC Index
try:
    nowcast_ticker = f"BENW{bloomberg_ctry_prefix}GC Index"
    bloomberg_nowcast = bb.get_series(nowcast_ticker)
    bloomberg_nowcast.plot(years_limit=3, tick_suffix="%", title=f"<b>{ctry_name} Bloomberg Nowcast</b>")
except Exception as e:
    display(Markdown(f"*Bloomberg nowcast not available for {ctry_name}*"))

### 1.3 GDP Growth & Components

In [ ]:
# Real GDP Growth - adapt ticker patterns per country
# This section needs country-specific ticker mapping
display(Markdown("**Note:** Populate with country-specific GDP tickers from Haver/Bloomberg"))

# Example structure:
# gdp_components = {
#     "Real GDP": "...",
#     "Consumption": "...",
#     "Investment": "...",
#     "Government": "...",
#     "Exports": "...",
#     "Imports": "...",
# }

### 1.4 Business Activity Surveys

In [ ]:
# PMIs - S&P Global Manufacturing/Services/Composite
# Common pattern: MPMI{ISO2}{MA/SA/CA} Index
try:
    pmi_tickers = {
        f"MPMI{bloomberg_ctry_prefix}MA Index": "Manufacturing PMI",
        f"MPMI{bloomberg_ctry_prefix}SA Index": "Services PMI",
        f"MPMI{bloomberg_ctry_prefix}CA Index": "Composite PMI",
    }
    
    pmi_collection = bb.create_collection(list(pmi_tickers.keys()))
    for i, title in enumerate(pmi_tickers.values()):
        pmi_collection[i].info.title = title
    
    pmi_collection.dashboard.table()
except Exception as e:
    display(Markdown(f"*PMI data not available: {e}*"))

In [ ]:
# PMI Charts
try:
    fig = pmi_collection.dashboard.plots(years_limit=3, hlines=50, show_0=True)
except:
    pass

### 1.5 Retail Sales & Consumer Activity

In [ ]:
# Retail Sales - country specific tickers
display(Markdown("**Note:** Add country-specific retail sales tickers"))

# Example patterns vary significantly by country
# US: "RSTAMOM Index"
# Canada: "CARSCHNG Index"
# etc.

### 1.6 Industrial Production & Manufacturing

In [ ]:
# Industrial Production
display(Markdown("**Note:** Add country-specific industrial production tickers"))

### 1.7 Growth Summary (AI-Generated)

In [ ]:
# AI Summary of Growth Conditions
# Combine all growth-related collections and get AI summary
try:
    # Example: growth_summary = iris.summarize(combined_growth_collection, bullets=5)
    # growth_summary.html()
    pass
except:
    pass

## 2. Prices: Inflation & Wages

**Key Questions:**
- What is the inflation rate and trend?
- How does it compare to central bank targets?
- What are expectations for future inflation?
- How are wages evolving?

### 2.1 Headline & Core Inflation

In [ ]:
# Core inflation measures - varies significantly by country
# US: CPI, PCE
# Eurozone: HICP
# Japan: CPI, Trimmed Mean, Mode
# etc.

display(Markdown("**Note:** Add country-specific inflation tickers (CPI, Core CPI, etc.)"))

### 2.2 Inflation Expectations & Surveys

In [ ]:
# Inflation expectations from surveys, breakevens, swaps
display(Markdown("**Note:** Add inflation expectation measures"))

### 2.3 Inflation Nowcasts & Forecasts

In [ ]:
# Cleveland Fed Nowcast, Bloomberg nowcasts, etc.
display(Markdown("**Note:** Add inflation nowcast tickers if available"))

### 2.4 Wage Growth & Labor Costs

In [ ]:
# Average hourly earnings, wage indices, unit labor costs
display(Markdown("**Note:** Add wage growth indicators"))

### 2.5 Inflation Components (Goods vs Services)

In [ ]:
# Breakdown: goods, services, energy, food
display(Markdown("**Note:** Add inflation component breakdowns"))

## 3. Slack: Output Gap & Capacity Utilization

**Key Questions:**
- Is the economy operating above or below potential?
- How much spare capacity exists?
- What does this imply for inflation pressures?

### 3.1 Output Gap Estimates

In [ ]:
# Output gap from central banks, IMF, OECD
# Example: Canada has B156IGAP@CANADA in Haver
display(Markdown("**Note:** Add output gap estimates if available"))

### 3.2 Capacity Utilization

In [ ]:
# Industrial capacity utilization rates
display(Markdown("**Note:** Add capacity utilization metrics"))

## 4. Employment: Labor Market Conditions

**Key Questions:**
- How tight is the labor market?
- What is the unemployment trend?
- Are there signs of labor market slack or tightness?

### 4.1 Unemployment Rate & Labor Force

In [ ]:
# Unemployment rate, labor force participation
display(Markdown("**Note:** Add unemployment and participation rate tickers"))

### 4.2 Employment Growth & Payrolls

In [ ]:
# Nonfarm payrolls (US), employment change metrics
display(Markdown("**Note:** Add employment change / payroll data"))

### 4.3 Jobless Claims & Leading Indicators

In [ ]:
# Initial/continuing claims, job openings
display(Markdown("**Note:** Add jobless claims and leading employment indicators"))

### 4.4 Job Openings & Labor Turnover

In [ ]:
# JOLTS (US), job vacancies, hires, quits, layoffs
display(Markdown("**Note:** Add job openings/turnover data if available"))

### 4.5 Alternative Employment Data

In [ ]:
# Indeed job postings, LinkUp, Homebase, ADP, etc.
display(Markdown("**Note:** Add alternative employment data sources"))

## 5. Financial Conditions

**Key Questions:**
- How tight or loose are financial conditions?
- What is happening with credit growth?
- Are there financial stability concerns?

### 5.1 Credit Growth & Debt Levels

In [ ]:
# Household debt, corporate debt, credit growth
display(Markdown("**Note:** Add credit growth and debt metrics"))

### 5.2 Financial Conditions Indices

In [ ]:
# FCI from Goldman, Bloomberg, central banks
display(Markdown("**Note:** Add financial conditions indices"))

### 5.3 Credit Spreads & Market Stress

In [ ]:
# Corporate spreads, sovereign spreads, stress indicators
display(Markdown("**Note:** Add credit spread and stress metrics"))

### 5.4 External Balance (Current Account, Trade)

In [ ]:
# Current account balance, trade balance
display(Markdown("**Note:** Add current account and trade data"))

## 6. Fiscal Conditions

**Key Questions:**
- What is the fiscal stance (surplus/deficit)?
- How sustainable is the debt trajectory?
- What is the government's fiscal policy outlook?

### 6.1 Budget Balance & Deficit

In [ ]:
# Government budget surplus/deficit
display(Markdown("**Note:** Add fiscal balance metrics"))

### 6.2 Public Debt Levels

In [ ]:
# Public debt as % of GDP
display(Markdown("**Note:** Add public debt metrics"))

### 6.3 Fiscal Policy Outlook

In [ ]:
# IMF projections, government forecasts
display(Markdown("**Note:** Add fiscal outlook and projections"))

---

## Summary & Next Steps

**Key Takeaways:**
- [To be filled based on data]

**Monitoring:**
- [Key metrics to watch]

**Risks:**
- [Upside and downside risks]

In [ ]:
Markdown(f"_Notebook updated at {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M')}_")